In [34]:
import pandas as pd

EXCEL_PATH = "data/portfolio.xlsx"

portfolio = pd.read_excel(EXCEL_PATH, sheet_name="Stock log")

Let's update Company Name, Market Price and Annual Dividend per Share
Notes:
- Market Price can be extracted from <ticker>.info["currentPrice"] for regular stocks, but not for ETFs
- For ETF there is <ticker>.info["navPrice"], which may slightly differ comparing to market price
- ETF net asset is a value of all assets of the fund, substracting liabilities. For ETFs NAV price is calculated every day
- Stocks dividends can be returned using dividendRate, but not for ETF. Ideally for ETFs we should calculate TTM dividend rate (Trailing twelwe months)
- yfinance does have <ticker>.info["trailingAnnualDividendRate"] but it's 0 for SCHD. Not reliable

We have to support both buy and sell operations
- Alghoritm would be to get dividends paid per each buy operation
- Sum all buy operations market value, dividends paid
- Subtract all sell operations (also calculate dividends paid per sell operation)
Sounds too expensive from operations point of view, but for small to medium log file with caching and rate limit coming soon it's ok.

In [35]:
from portfolio.base import TickerData
from main import (
    calculate_etf_ttm,
    calculate_percentage_difference,
    dividends_paid_per_share,
    calculate_total_return
)

ticker = TickerData()

for i in range(len(portfolio)):
  ticker_data = ticker.get_ticker_data(portfolio.loc[i, "Ticker"])
  match ticker_data.info["quoteType"]:
    case "ETF":
      annual_dividend_per_share = calculate_etf_ttm(ticker_data)
      market_price = round(ticker_data.info["navPrice"], 2)
    case _:
      annual_dividend_per_share = ticker_data.info["dividendRate"]
      market_price = round(ticker_data.info["currentPrice"],2)

  buy_price = portfolio.loc[i, "Buy Price"]
  capital_gains = calculate_percentage_difference(market_price,buy_price)
  num_of_shares = portfolio.loc[i, "Number of Shares"]
  sum_of_divs_per_share = dividends_paid_per_share(portfolio.loc[i, "Date of Purchase"],ticker_data)
  total_return = calculate_total_return(sum_of_divs_per_share,buy_price,market_price)

  portfolio.loc[i, "Company Name"] = ticker_data.info["shortName"]
  portfolio.loc[i, "Market Price"] = market_price
  portfolio.loc[i, "Annual Dividend per Share"] = annual_dividend_per_share
  portfolio.loc[i, "Capital Gains"] = round(capital_gains, 2)
  portfolio.loc[i, "Dividends Paid"] = sum_of_divs_per_share * num_of_shares
  portfolio.loc[i, "Total Return"] = total_return



In [54]:
for i in range(len(portfolio)):
    cost_basis = portfolio.loc[i, "Number of Shares"] * portfolio.loc[i, "Buy Price"]
    market_value = portfolio.loc[i, "Number of Shares"] * portfolio.loc[i, "Market Price"]
    portfolio.loc[i, "Cost Basis"] = cost_basis
    portfolio.loc[i, "Market Value"] = market_value
    

In [62]:
## Update fields if operation is sell

portfolio.loc[portfolio["Operation"] == "Sell", "Number of Shares"] = portfolio["Number of Shares"] * -1
portfolio.loc[portfolio["Operation"] == "Sell", "Cost Basis"] = portfolio["Cost Basis"] * -1
portfolio.loc[portfolio["Operation"] == "Sell", "Market Value"] = portfolio["Market Value"] * -1
portfolio.loc[portfolio["Operation"] == "Sell", "Dividends Paid"] = portfolio["Dividends Paid"] * -1

In [63]:
def test_some_pandas(market,paid):
    percentage_difference = ((paid - market) / abs(market)) * 100
    return round(percentage_difference,2)

summary = pd.pivot_table(
    portfolio,
    index="Ticker",
    margins=False,
    margins_name='Total',
    values=["Number of Shares", "Cost Basis", "Market Value","Dividends Paid"],
    aggfunc="sum",
    sort=False
)

average_price_paid_per_share = round(summary["Cost Basis"] / summary["Number of Shares"], 2)
cost_basis_and_dividends = summary["Cost Basis"] - summary["Dividends Paid"]
average_price_paid_per_share_including_dividends = round(cost_basis_and_dividends / summary["Number of Shares"], 2)
average_price_market_per_share = round(summary["Market Value"] / summary["Number of Shares"], 2)
capital_gains = round(calculate_percentage_difference(average_price_market_per_share,average_price_paid_per_share), 2)
total_return = round(calculate_percentage_difference(average_price_market_per_share,average_price_paid_per_share_including_dividends), 2)

summary["Average price paid, USD"] = average_price_paid_per_share
summary["Capital Gains, %"] = capital_gains
summary["Total Return, %"] = total_return
summary

,Number of Shares,Cost Basis,Market Value,Dividends Paid,"Average price paid, USD","Capital Gains, %","Total Return, %"
Ticker,,,,,,,
AY,987,23742.85,21417.90,923.43,24.06,-9.81,-6.14
SCHD,983,74363.95,74934.09,1376.20,75.65,0.77,2.67


In [58]:
portfolio.head(5)

,Company Name,Ticker,Operation,Number of Shares,Buy Price,Market Price,Annual Dividend per Share,Date of Purchase,Cost Basis,Market Value,Capital Gains,Dividends Paid,Total Return
0,Atlantica Sustainable Infrastru,AY,Buy,687,23.55,21.70,1.78,22.06.2023,16178.85,14907.90,-7.86,611.43,-4.24
1,Atlantica Sustainable Infrastru,AY,Buy,400,24.66,21.70,1.78,27.07.2023,9864.00,8680.00,-12.00,356.00,-8.71
2,Schwab US Dividend Equity ETF,SCHD,Buy,983,75.65,76.23,2.66,27.07.2023,74363.95,74934.09,0.77,1376.20,2.67
3,Atlantica Sustainable Infrastru,AY,Sell,-100,23.00,21.70,1.78,25.11.2023,-2300.00,-2170.00,-5.65,-44.00,-3.81


In [46]:
with pd.ExcelWriter(EXCEL_PATH) as writer:
    portfolio.to_excel(writer, sheet_name="Stock log", index=False)
    summary.to_excel(writer, sheet_name="Portfolio Summary", index=True)